In [13]:
import os
os.getcwd()

'C:\\Users\\Daniel\\RNN'

In [14]:
from selenium.webdriver import Chrome

browser = Chrome()

url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d1371740-Reviews-Mugyodong_Bugeokukjib-Seoul.html'

browser.get(url)

In [15]:
more_links = browser.find_elements_by_css_selector(".ulBlueLinks")

In [16]:
for link in more_links:
    try:
        link.click()
    except:
        pass

In [17]:
# find reviews
reviews = browser.find_elements_by_css_selector(".review-container")

In [18]:
reviews

[<selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-11")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-12")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-13")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-14")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-15")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-16")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3e5", element="0.6562511506860491-17")>,
 <selenium.webdriver.remote.webelement.WebElement (session="965775fb0d68d3ce02e3e0679e59c3

In [19]:
s = len("ui_bubble_rating_bubble_")

review_list = []
for review in reviews:
    # find rating score
    rating = review.find_element_by_css_selector(".ui_bubble_rating")
    
    # get the class of rating with using get_attribute()
    cls = rating.get_attribute("class")
    
    # extract the "s" part and convert its type into int
    score = int(cls[s:])
    
    # find the reviews
    comment = review.find_element_by_css_selector(".partial_entry")
    
    review_list.append((score, comment.text))

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame(review_list, columns = ["score", "text"])
df

,score,text
0,40,북어국집이 있는 작은 골목을 지날때마다 많은 사람들이 줄서있는 것을 보면...\n'...
1,50,왜 미리 몰랐을까 라고 후회 했던 집\n정말 맛보시길 추천 합니다\n\n일요일 오전...
2,50,"""숙취에 최고의 음식일 듯, 유명한 만큼 맛았고 친절하고 회전이 빠른 식당""\n""직..."
3,50,변함없늗 속풀이. 깔끔하고 회전 빠르고 가격도 주변에 비해 착함. 포장도 맛있음. ...
4,40,엄청난 맛집은 아니지만 나쁘지 않음으로 꾸준히 유지가 되는 집. 너무 큰 기대를 가...
5,50,"단일 품목으로 오랫동안 손님들의 사랑을 듬뿍 받는 이곳, 5 년만에 다시 들렀어요...."
6,40,"첫느낌은 노포에서 맛볼 수 있는 북어국이 아닐까 했는데,들어가보니\n전혀 반대입니다..."
7,50,이집은 정말 대한민국에서 최고의 해장 북엉국집이라 인정해요\n\n어제 과음으로 힘뜰...
8,40,"짭자름한 북엇국물에 밥 한 공기면, 전날 숙취를 깔끔히 떨어낼 수 있다.\n가격도 ..."
9,30,명동에 명동교자가 있다면 무교동엔 북어국집이 있습니다..\n오래된 곳이라는 느낌이지...


In [22]:
df.to_csv("tripadvisor.csv", encoding = "utf-8")

In [23]:
# RNN
import keras

C:\Users\Daniel\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
from keras.datasets import imdb

In [25]:
data = imdb.load_data()

17465344/17464789 [==============================] - 85s 5us/step


In [26]:
(X_train, y_train), (X_test, y_test) = data

In [27]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [28]:
y_train[0]

1

In [29]:
# RNN is needed to be designated max length : 
MAXLEN = 20

In [30]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [32]:
X_train_seq = pad_sequences(X_train, MAXLEN)
X_test_seq = pad_sequences(X_test, MAXLEN)

In [33]:
NUM_WORDS = X_train_seq.max() + 1

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

In [35]:
ff = Sequential()
ff.add(Embedding(input_dim = NUM_WORDS, output_dim = 8, input_length = MAXLEN))
ff.add(Flatten())
ff.add(Dense(1, activation = "sigmoid"))

In [36]:
ff.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             708632    
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 708,793
Trainable params: 708,793
Non-trainable params: 0
_________________________________________________________________


In [37]:
from keras.optimizers import RMSprop

In [38]:
ff.compile(optimizer = RMSprop(), loss = "binary_crossentropy", metrics = ["acc"])

In [39]:
from keras.callbacks import EarlyStopping

In [41]:
history = ff.fit(
    X_train_seq,
    y_train,
    epochs = 30,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = [EarlyStopping(monitor = "val_acc")])

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 2s 104us/step - loss: 0.6889 - acc: 0.5582 - val_loss: 0.6811 - val_acc: 0.6316
Epoch 2/30
20000/20000 [==============================] - 2s 92us/step - loss: 0.6514 - acc: 0.7302 - val_loss: 0.6390 - val_acc: 0.6908
Epoch 3/30
20000/20000 [==============================] - 2s 103us/step - loss: 0.5820 - acc: 0.7631 - val_loss: 0.5852 - val_acc: 0.7114
Epoch 4/30
20000/20000 [==============================] - 2s 92us/step - loss: 0.5115 - acc: 0.7893 - val_loss: 0.5459 - val_acc: 0.7270
Epoch 5/30
20000/20000 [==============================] - 2s 90us/step - loss: 0.4549 - acc: 0.8129 - val_loss: 0.5210 - val_acc: 0.7388
Epoch 6/30
20000/20000 [==============================] - 2s 96us/step - loss: 0.4103 - acc: 0.8337 - val_loss: 0.5066 - val_acc: 0.7446
Epoch 7/30
20000/20000 [==============================] - 2s 92us/step - loss: 0.3734 - acc: 0.8515 - val_loss: 0.4982 - val_ac

In [42]:
rnn = Sequential()

In [43]:
from keras.layers import LSTM

In [44]:
rnn.add(Embedding(input_dim = NUM_WORDS, output_dim = 8, input_length = MAXLEN))
rnn.add(LSTM(1, activation = "sigmoid", return_sequences = False))

In [45]:
rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             708632    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1)                 40        
Total params: 708,672
Trainable params: 708,672
Non-trainable params: 0
_________________________________________________________________


In [46]:
rnn.compile(optimizer = RMSprop(), loss = "binary_crossentropy", metrics = ["accuracy"])

In [49]:
rnn.fit(
    X_train_seq,
    y_train,
    epochs = 30,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = [EarlyStopping(monitor = "val_acc")])

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 4s 206us/step - loss: 0.7316 - acc: 0.4985 - val_loss: 0.7005 - val_acc: 0.5062
Epoch 2/30
20000/20000 [==============================] - 3s 165us/step - loss: 0.6800 - acc: 0.5451 - val_loss: 0.6659 - val_acc: 0.5994
Epoch 3/30
20000/20000 [==============================] - 4s 180us/step - loss: 0.6487 - acc: 0.6621 - val_loss: 0.6481 - val_acc: 0.6374
Epoch 4/30
20000/20000 [==============================] - 3s 160us/step - loss: 0.6237 - acc: 0.7046 - val_loss: 0.6306 - val_acc: 0.6538
Epoch 5/30
20000/20000 [==============================] - 3s 153us/step - loss: 0.5984 - acc: 0.7272 - val_loss: 0.6147 - val_acc: 0.6686
Epoch 6/30
20000/20000 [==============================] - 3s 156us/step - loss: 0.5746 - acc: 0.7435 - val_loss: 0.6032 - val_acc: 0.6814
Epoch 7/30
20000/20000 [==============================] - 3s 154us/step - loss: 0.5556 - acc: 0.7500 - val_loss: 0.5962 - v

In [50]:
# add dense layer
rnn2 = Sequential()
rnn2.add(Embedding(input_dim = NUM_WORDS, output_dim = 8, input_length = MAXLEN))
rnn2.add(LSTM(32, activation = "tanh", return_sequences = False))
rnn2.add(Dense(1, activation = "sigmoid"))

In [51]:
rnn2.compile(optimizer =RMSprop(), loss = "binary_crossentropy", metrics = ["accuracy"])

In [52]:
rnn2.fit(
    X_train_seq,
    y_train,
    epochs = 30,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = [EarlyStopping(monitor ="val_acc")])

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 5s 248us/step - loss: 0.6261 - acc: 0.6397 - val_loss: 0.5625 - val_acc: 0.7090
Epoch 2/30
20000/20000 [==============================] - 4s 192us/step - loss: 0.4792 - acc: 0.7744 - val_loss: 0.5128 - val_acc: 0.7434
Epoch 3/30
20000/20000 [==============================] - 4s 197us/step - loss: 0.4057 - acc: 0.8189 - val_loss: 0.5041 - val_acc: 0.7544
Epoch 4/30
20000/20000 [==============================] - 4s 205us/step - loss: 0.3594 - acc: 0.8460 - val_loss: 0.5045 - val_acc: 0.7594
Epoch 5/30
20000/20000 [==============================] - 4s 203us/step - loss: 0.3269 - acc: 0.8605 - val_loss: 0.5455 - val_acc: 0.7534


In [53]:
# aggregaiing all outputs of every RNN nodes to dense layer
rnn3 = Sequential()
rnn3.add(Embedding(input_dim = NUM_WORDS, output_dim = 8, input_length = MAXLEN))
rnn3.add(LSTM(4, activation = "tanh", return_sequences = True))
rnn3.add(Flatten())
rnn3.add(Dense(1, activation = "sigmoid"))

In [54]:
rnn3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 8)             708632    
_________________________________________________________________
lstm_3 (LSTM)                (None, 20, 4)             208       
_________________________________________________________________
flatten_2 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 81        
Total params: 708,921
Trainable params: 708,921
Non-trainable params: 0
_________________________________________________________________


In [55]:
rnn3.compile(optimizer = RMSprop(), loss = "binary_crossentropy", metrics = ["accuracy"])

In [56]:
rnn3.fit(
    X_train_seq,
    y_train,
    epochs = 30, 
    batch_size = 128,
    validation_split = 0.2,
    callbacks = [EarlyStopping(monitor = "val_acc")])

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 7s 333us/step - loss: 0.6767 - acc: 0.5933 - val_loss: 0.6352 - val_acc: 0.6400
Epoch 2/30
20000/20000 [==============================] - 5s 229us/step - loss: 0.5532 - acc: 0.7236 - val_loss: 0.5735 - val_acc: 0.7032
Epoch 3/30
20000/20000 [==============================] - 4s 213us/step - loss: 0.4638 - acc: 0.7832 - val_loss: 0.5587 - val_acc: 0.7114
Epoch 4/30
20000/20000 [==============================] - 4s 206us/step - loss: 0.4098 - acc: 0.8138 - val_loss: 0.5634 - val_acc: 0.7214
Epoch 5/30
20000/20000 [==============================] - 4s 215us/step - loss: 0.3709 - acc: 0.8362 - val_loss: 0.5499 - val_acc: 0.7258
Epoch 6/30
20000/20000 [==============================] - 5s 225us/step - loss: 0.3403 - acc: 0.8556 - val_loss: 0.5692 - val_acc: 0.7340
Epoch 7/30
20000/20000 [==============================] - 5s 228us/step - loss: 0.3155 - acc: 0.8678 - val_loss: 0.5614 - v